In [71]:
import pandas as pd
from IPython.display import display, display_html
import os.path
import sqlite3

lyrics_arc = 'lyrics-data.csv'
lyrics_arc_fltr = 'lyrics_archive_filtered.csv'

conn = sqlite3.connect('chinook.db')

# Verifica se o arquivo existe, filtra músicas da banda "AC/DC"
# e gera arquivo limpo
def filter_csv():
    if os.path.exists(lyrics_arc) is True:
        df = pd.read_csv(lyrics_arc, encoding='utf8')
        composer = df[df['ALink'].eq('/ac-dc/')]
        composer.to_csv(lyrics_arc_fltr, index=False)

# Cria tabela para receber os dados
def create_table():
    try:
        conn.execute("""CREATE TABLE IF NOT EXISTS lyrics
                     (TrackId INTEGER PRIMARY KEY,
                      Name NVARCHAR(255) NOT NULL UNIQUE,
                      Lyric TEXT NOT NULL)""")
    except:
        pass

# Insere os dados na tabela
def insert_data():
    if os.path.exists(lyrics_arc_fltr) is True:
        df = pd.read_csv(lyrics_arc_fltr, encoding='utf8')
        df = df[['SName', 'Lyric']]
        df = df.rename(columns={'SName': 'Name'})

        cur = conn.cursor()
        insert_plSql = '''INSERT INTO lyrics (Name, Lyric) VALUES (?, ?);'''
        select_plSql = '''SELECT TrackId FROM lyrics WHERE Name=?'''

        for i, x in df.iterrows():
            cur.execute(select_plSql, (tuple(x)[0],))

            if cur.fetchone() is None:
                cur.execute(insert_plSql, tuple(x))

            conn.commit()

# Gera o dataframe com os 3 gêneros musicais mais vendidos no Brasil
def top3_genres_brasil():
    strSql = """SELECT D.Name AS GENERO,
                 (A.UnitPrice * SUM (A.Quantity)) AS VL_TOTAL
                 FROM invoice_items AS A
                 LEFT JOIN invoices AS B ON B.InvoiceId = A.InvoiceId
                 LEFT JOIN tracks AS C ON C.TrackId = A.TrackId
                 LEFT JOIN genres AS D ON D.GenreId = C.GenreId
                 WHERE B.BillingCountry = 'Brazil'
                 GROUP BY D.Name
                 ORDER BY VL_TOTAL DESC LIMIT 3"""
    df = pd.read_sql_query(strSql, conn)
    print('\033[1m' + 'TOP3 GÊNEROS MUSICAIS MAIS VENDIDOS (BRASIL)')   
    display(df)

# Gera o dataframe com as vendas por música do album "Mais do Mesmo"
def mais_do_mesmo():
    strSql = """SELECT B.Name AS MUSICA,
                 IFNULL(SUM (C.Quantity), 0) AS QTDE_TOTAL,
                 IFNULL(SUM (C.UnitPrice), 0) AS VL_TOTAL
                 FROM albums AS A
                 LEFT JOIN tracks AS B ON B.AlbumId = A.AlbumId
                 LEFT JOIN invoice_items AS C ON C.TrackId = B.TrackId
                 WHERE A.Title = 'Mais Do Mesmo'
                 GROUP BY A.Title, B.Name
                 ORDER BY VL_TOTAL DESC, QTDE_TOTAL DESC"""
    df = pd.read_sql_query(strSql, conn)
    print('\033[1m' + 'TOTAL DE VENDAS DO ALBUM "MAIS DO MESMO"')
    display(df)


filter_csv()
create_table()
insert_data()
top3_genres_brasil()
mais_do_mesmo()
conn.close()